In [1]:
import pandas as pd

### Carga de datos a unir

In [2]:
source_country_electricity_consumption = pd.read_csv('source_country_electricity_consumption.csv')
source_country_electricity_energyCO2 = pd.read_csv('source_country_electricity_energyCO2.csv')
population_country_electricity_consumption = pd.read_csv('population_country_electricity_consumption.csv')
owid_co2_data = pd.read_csv('../rawDataset/owid-co2-data.csv')
country = pd.read_csv('../Modelo_ER/country.csv')

In [3]:
source_country_electricity_consumption

,country,year,source,consumptionElectricity,productionElectricity,energyPerCapita,elecPerCapita
0,Afghanistan,1980,coal,0.0,0.0,0.0,0.0
1,Afghanistan,1980,gas,0.0,0.0,0.0,0.0
2,Afghanistan,1980,hydro,0.0,0.0,0.0,0.0
3,Afghanistan,1980,nuclear,0.0,0.0,0.0,0.0
4,Afghanistan,1980,oil,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
79573,Zimbabwe,2021,oil,0.0,0.0,0.0,0.0
79574,Zimbabwe,2021,other_renewables,0.0,0.0,0.0,0.0
79575,Zimbabwe,2021,renewables,0.0,0.0,0.0,0.0
79576,Zimbabwe,2021,solar,0.0,0.0,0.0,0.0


### Tabla ***source_country_electricity***

#### Segmentación de la vista a reemplazar con los valores del csv emissionsCO2

In [4]:
source_country_electricity_consumption_view = source_country_electricity_consumption[(source_country_electricity_consumption['consumptionElectricity'] == 0) & (source_country_electricity_consumption['productionElectricity'] == 0)]
source_country_electricity_consumption_view = source_country_electricity_consumption_view[source_country_electricity_consumption_view.source.isin(source_country_electricity_energyCO2.source.unique())]
source_country_electricity_consumption_view = source_country_electricity_consumption_view[['country','year','source','consumptionElectricity','productionElectricity']]
source_country_electricity_consumption_view

,country,year,source,consumptionElectricity,productionElectricity
0,Afghanistan,1980,coal,0.0,0.0
1,Afghanistan,1980,gas,0.0,0.0
3,Afghanistan,1980,nuclear,0.0,0.0
4,Afghanistan,1980,oil,0.0,0.0
5,Afghanistan,1980,other_renewables,0.0,0.0
...,...,...,...,...,...
79569,Zimbabwe,2021,coal,0.0,0.0
79570,Zimbabwe,2021,gas,0.0,0.0
79572,Zimbabwe,2021,nuclear,0.0,0.0
79573,Zimbabwe,2021,oil,0.0,0.0


#### DataFrame auxiliar, cuyo valor será enviado al DataFrame original reemplazando valores

In [5]:
aux = source_country_electricity_consumption_view.merge(source_country_electricity_energyCO2, on=['country', 'year', 'source'], how = 'left').fillna(0)
aux.drop(['consumptionElectricity_x', 'productionElectricity_x'], axis = 1, inplace = True)
aux.columns = ['country','year','source','consumptionElectricity','productionElectricity']
aux

,country,year,source,consumptionElectricity,productionElectricity
0,Afghanistan,1980,coal,0.727,0.690
1,Afghanistan,1980,gas,0.614,18.411
2,Afghanistan,1980,nuclear,0.000,0.000
3,Afghanistan,1980,oil,4.286,0.000
4,Afghanistan,1980,other_renewables,2.165,2.165
...,...,...,...,...,...
27401,Zimbabwe,2021,coal,0.000,0.000
27402,Zimbabwe,2021,gas,0.000,0.000
27403,Zimbabwe,2021,nuclear,0.000,0.000
27404,Zimbabwe,2021,oil,0.000,0.000


#### Cohesión de valores de columnas

In [6]:
source_country_electricity_consumption_view[:] = aux.values

In [7]:
source_country_electricity_consumption_view

,country,year,source,consumptionElectricity,productionElectricity
0,Afghanistan,1980,coal,0.727,0.690
1,Afghanistan,1980,gas,0.614,18.411
3,Afghanistan,1980,nuclear,0.000,0.000
4,Afghanistan,1980,oil,4.286,0.000
5,Afghanistan,1980,other_renewables,2.165,2.165
...,...,...,...,...,...
79569,Zimbabwe,2021,coal,0.000,0.000
79570,Zimbabwe,2021,gas,0.000,0.000
79572,Zimbabwe,2021,nuclear,0.000,0.000
79573,Zimbabwe,2021,oil,0.000,0.000


In [8]:
index = source_country_electricity_consumption_view.index.to_list()
def f_merge(s):
    if s.name in index:
        s['consumptionElectricity'] = source_country_electricity_consumption_view.loc[s.name, 'consumptionElectricity']
        s['productionElectricity'] = source_country_electricity_consumption_view.loc[s.name, 'productionElectricity']
    return s

In [9]:
source_country_electricity_consumption = source_country_electricity_consumption.apply(f_merge, axis = 1)

In [10]:
source_country_electricity_consumption

,country,year,source,consumptionElectricity,productionElectricity,energyPerCapita,elecPerCapita
0,Afghanistan,1980,coal,0.727,0.690,0.0,0.0
1,Afghanistan,1980,gas,0.614,18.411,0.0,0.0
2,Afghanistan,1980,hydro,0.000,0.000,0.0,0.0
3,Afghanistan,1980,nuclear,0.000,0.000,0.0,0.0
4,Afghanistan,1980,oil,4.286,0.000,0.0,0.0
...,...,...,...,...,...,...,...
79573,Zimbabwe,2021,oil,0.000,0.000,0.0,0.0
79574,Zimbabwe,2021,other_renewables,0.000,0.000,0.0,0.0
79575,Zimbabwe,2021,renewables,0.000,0.000,0.0,0.0
79576,Zimbabwe,2021,solar,0.000,0.000,0.0,0.0


In [11]:
source_country_electricity_consumption.sort_values(by=['country','year','source']).reset_index(drop=True).to_csv('../Modelo_ER/source_country_electricity.csv', index=False)

### Tabla ***population_country_electricity***

In [12]:
countries = country.country.values


In [13]:
owid_co2_data = owid_co2_data.replace('Micronesia (country)','Micronesia')
owid_co2_data = owid_co2_data.replace('Timor','Timor-Leste')
owid_co2_data = owid_co2_data.replace('Guinea-Bissau','Guinea Bissau')

In [14]:
## regiones
owid_co2_regions = owid_co2_data[~owid_co2_data['country'].isin(countries)]

In [15]:
owid_co2_country = owid_co2_data[owid_co2_data['country'].isin(countries)]
owid_co2_country

,iso_code,country,year,co2,co2_per_capita,trade_co2,cement_co2,cement_co2_per_capita,coal_co2,coal_co2_per_capita,...,ghg_excluding_lucf_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,AFG,Afghanistan,1949,0.015,0.002,NaN,NaN,NaN,0.015,0.002,...,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1950,0.084,0.011,NaN,NaN,NaN,0.021,0.003,...,NaN,NaN,NaN,NaN,NaN,7752117.0,9.421400e+09,NaN,NaN,NaN
2,AFG,Afghanistan,1951,0.092,0.012,NaN,NaN,NaN,0.026,0.003,...,NaN,NaN,NaN,NaN,NaN,7840151.0,9.692280e+09,NaN,NaN,NaN
3,AFG,Afghanistan,1952,0.092,0.012,NaN,NaN,NaN,0.032,0.004,...,NaN,NaN,NaN,NaN,NaN,7935996.0,1.001733e+10,NaN,NaN,NaN
4,AFG,Afghanistan,1953,0.106,0.013,NaN,NaN,NaN,0.038,0.005,...,NaN,NaN,NaN,NaN,NaN,8039684.0,1.063052e+10,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25984,ZWE,Zimbabwe,2016,10.738,0.765,1.415,0.639,0.046,6.959,0.496,...,2.076,11.50,0.820,6.21,0.443,14030338.0,2.096179e+10,46.666,3326.073,2.226
25985,ZWE,Zimbabwe,2017,9.582,0.673,1.666,0.678,0.048,5.665,0.398,...,2.023,11.62,0.816,6.35,0.446,14236599.0,2.194784e+10,45.936,3226.617,2.093
25986,ZWE,Zimbabwe,2018,11.854,0.821,1.308,0.697,0.048,7.101,0.492,...,2.173,11.96,0.828,6.59,0.456,14438812.0,2.271535e+10,47.502,3289.887,2.091
25987,ZWE,Zimbabwe,2019,10.949,0.748,1.473,0.697,0.048,6.020,0.411,...,NaN,NaN,NaN,NaN,NaN,14645473.0,NaN,49.427,3374.877,NaN


In [16]:
owid_co2_country = owid_co2_country[['country','year','co2_per_capita']]
owid_co2_country = owid_co2_country.rename(columns={'co2_per_capita':'co2PerCapita'})

In [17]:
owid_co2_country.fillna(0, inplace=True)

In [18]:
population_country_electricity = population_country_electricity_consumption.merge(owid_co2_country, on=['country','year'], how='left').fillna(0)

In [19]:
population_country_electricity

,country,year,electricityDemand,electricityGeneration,population,co2PerCapita
0,Afghanistan,1980,0.00,0.00,13356500,0.132
1,Afghanistan,1981,0.00,0.00,13171679,0.150
2,Afghanistan,1982,0.00,0.00,12882518,0.163
3,Afghanistan,1983,0.00,0.00,12537732,0.201
4,Afghanistan,1984,0.00,0.00,12204306,0.231
...,...,...,...,...,...,...
8837,Zimbabwe,2017,9.53,7.31,14236599,0.673
8838,Zimbabwe,2018,10.15,9.13,14438812,0.821
8839,Zimbabwe,2019,12.42,11.31,14645473,0.748
8840,Zimbabwe,2020,12.22,11.11,14862927,0.709


In [20]:
population_country_electricity.sort_values(by=['country','year']).reset_index(drop=True).to_csv('../Modelo_ER/population_country_electricity.csv', index=False)